In [1]:
import tensorflow as tf

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [58]:
tf.reset_default_graph()

h, w = 128, 128
img = tf.placeholder(dtype=tf.float32, shape=[None, h, w, 1])
img_scaled = tf.placeholder(dtype=tf.float32, shape=[None, 112, 112, 1])

# LOW-LEVEL FEATURES: FIXED
fconv2d_11 = tf.layers.conv2d(inputs=img_scaled, filters=64, kernel_size=3, strides=2, activation=tf.nn.relu, padding='same', reuse=None, name='lf_conv2d_11')
fconv2d_12 = tf.layers.conv2d(inputs=fconv2d_11, filters=128, kernel_size=3, strides=1, activation=tf.nn.relu, padding='same', reuse=None, name='lf_conv2d_12')

fconv2d_21 = tf.layers.conv2d(inputs=fconv2d_12, filters=128, kernel_size=3, strides=2, activation=tf.nn.relu, padding='same', reuse=None, name='lf_conv2d_21')
fconv2d_22 = tf.layers.conv2d(inputs=fconv2d_21, filters=256, kernel_size=3, strides=1, activation=tf.nn.relu, padding='same', reuse=None, name='lf_conv2d_22')

fconv2d_31 = tf.layers.conv2d(inputs=fconv2d_22, filters=256, kernel_size=3, strides=2, activation=tf.nn.relu, padding='same', reuse=None, name='lf_conv2d_31')
fconv2d_32 = tf.layers.conv2d(inputs=fconv2d_31, filters=512, kernel_size=3, strides=1, activation=tf.nn.relu, padding='same', reuse=None, name='lf_conv2d_32')

# GLOBAL FEATURE NETWORK
gconv2d_1 = tf.layers.conv2d(inputs=fconv2d_32, filters=512, kernel_size=3, strides=2, activation=tf.nn.relu, padding='same', name='gf_conv2d_1')
gconv2d_2 = tf.layers.conv2d(inputs=gconv2d_1, filters=512, kernel_size=3, strides=1, activation=tf.nn.relu, padding='same', name='gf_conv2d_2')
gconv2d_3 = tf.layers.conv2d(inputs=gconv2d_2, filters=512, kernel_size=3, strides=2, activation=tf.nn.relu, padding='same', name='gf_conv2d_3')
gconv2d_4 = tf.layers.conv2d(inputs=gconv2d_3, filters=512, kernel_size=3, strides=1, activation=tf.nn.relu, padding='same', name='gf_conv2d_4')
gfc_1 = tf.layers.dense(inputs=gconv2d_4, units=1024, activation=tf.nn.relu, name='gf_fc_1')
gfc_2 = tf.layers.dense(inputs=gfc_1, units=512, activation=tf.nn.relu, name='gf_fc_2')
gfc_3 = tf.layers.dense(inputs=gfc_2, units=256, activation=tf.nn.relu, name='gf_fc_3')

# LOW-LEVEL FEATURES: VARIABLE
vconv2d_11 = tf.layers.conv2d(inputs=img, filters=64, kernel_size=3, strides=2, activation=tf.nn.relu, padding='same', reuse=True, name='lf_conv2d_11')
vconv2d_12 = tf.layers.conv2d(inputs=vconv2d_11, filters=128, kernel_size=3, strides=1, activation=tf.nn.relu, padding='same', reuse=True, name='lf_conv2d_12')

vconv2d_21 = tf.layers.conv2d(inputs=vconv2d_12, filters=128, kernel_size=3, strides=2, activation=tf.nn.relu, padding='same', reuse=True, name='lf_conv2d_21')
vconv2d_22 = tf.layers.conv2d(inputs=vconv2d_21, filters=256, kernel_size=3, strides=1, activation=tf.nn.relu, padding='same', reuse=True, name='lf_conv2d_22')

vconv2d_31 = tf.layers.conv2d(inputs=vconv2d_22, filters=256, kernel_size=3, strides=2, activation=tf.nn.relu, padding='same', reuse=True, name='lf_conv2d_31')
vconv2d_32 = tf.layers.conv2d(inputs=vconv2d_31, filters=512, kernel_size=3, strides=1, activation=tf.nn.relu, padding='same', reuse=True, name='lf_conv2d_32')

# MID-LEVEL FEATURE NETWORK
mconv2d_1 = tf.layers.conv2d(inputs=vconv2d_32, filters=512, kernel_size=3, strides=1, activation=tf.nn.relu, padding='same', name='mf_conv2d_1')
mconv2d_2 = tf.layers.conv2d(inputs=mconv2d_1, filters=256, kernel_size=3, strides=1, activation=tf.nn.relu, padding='same', name='mf_conv2d_2')



In [59]:
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    file_writer = tf.summary.FileWriter('', sess.graph)